<div>
<img src="https://www.nebrija.com/images/logos/logotipo-universidad-nebrija.jpg" width="200">
</div>

**MODELOS DE PROGRAMACION: MODELO ADIABATICO** -
Prof: Carmen Pellicer Lostao

# Constrained Scheduling

## Intro

In this Notebook we will formulate an optimization problem with real variables. Programatically we will use the [Constrained Quadratic Model](https://docs.ocean.dwavesys.com/en/latest/concepts/cqm.html) (DQM) class in Ocean tools to formulate it.

Then we will use DWave Hybrid solvers to sample the DQM object of our optimization problem. Specifically we will use the [LeapHybridDQMSampler()](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html#id19) and the SAPI to sumbmit the sampling job to the QPU

We install DWave Ocean Libraries and setup them.

In [ ]:
!pip install dwave-ocean-sdk

In [ ]:
!dwave setup

In [ ]:
!dwave config inspect

## Discrete Quadratic Models


The discrete quadratic model (DQM) is a polynomial over discrete variables, with all terms of degree two or less, where a discrete variable represents some collection of distinct values, such as {red, green, blue, yellow} or {3.2, 67}, which are called the variable’s cases.

A discrete quadratic model may be defined as:

$H(d) = \sum_{i=1}^M a_i d_i + \sum_{i=1}^M \sum_{j=1}^M b_{i,j} (d_i d_j)+c$

where $d_i$ are the discrete variables,
 and $a_i$ and $b_{i,j}$ are real-valued functions, and $c$ is a constant (offset).

You can represent any DQM with an equivalent model over binary variables by replacing each discrete variable, $d_i$, with a vector of binary variables using [one-hot encoding](https://en.wikipedia.org/wiki/One-hot), where exactly one binary variable is True and all others are False: $\sum_{i=1}^N x_{i,a} =1 \forall i$

In particular, a discrete quadratic model for $M$ discrete variables,$d_i$, each with $n_i$ cases, is then defined by using a binary variable, $x_{i,u}$, to indicate whether discrete variable $d_i$ is set to case $u$. The objective function can be expressed by the equation:

$H(x) = \sum_{i=1}^M \sum_{u=1}^{n_i} a_{i,u} x_{i,u} + \sum_{i=1}^M \sum_{j=1}^M \sum_{u=1}^{n_i} \sum_{v=1}^{n_i} b_{i,j,u.v}x_{i,u}+c$

Both representations are equivalent over the feasible space; that is, the solutions that meet the one-hot-encoding constraints. The second representation ascribes energies both to the feasible space (satisfying constraints), and an infeasible space (violating constraints). The second representation is used by the [Disctrete Quadratic Model](https://docs.ocean.dwavesys.com/en/latest/concepts/dqm.html) (DQM) class in Ocean tools

## CSP Employee scheduling

Building a schedule for employees can be an extremely complex optimization problem in which managers must balance employee preferences against schedule requirements. In this example, we show how a discrete quadratic model (DQM) can be used to model this problem and how the hybrid solvers available in DWave Leap can optimize over these competing scheduling and preference needs.

First we import the libraries needed in the notebook

In [1]:
from dimod import DiscreteQuadraticModel
from dwave.system import LeapHybridDQMSampler
import numpy as np
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt

To make the example more felxible we ask the user to provide the input values of the number of employees $M$ and the number of shifts $N$.

Note that we should have at least as many employees as there are shifts.

In [2]:
# Collect user input on size of problem
response_1 = input("\nEnter number of employees > ")
try:
    num_employees = int(response_1)
except ValueError:
    print("Must input an integer.")
    num_employees = int(input("\nEnter number of employees > "))

response_2 = input("\nEnter number of shifts > ")
try:
    num_shifts = int(response_2)
except ValueError:
    print("Must input an integer.")
    num_shifts = int(input("\nEnter number of shifts > "))

if num_employees < num_shifts:
    print("\n**Number of employees must be at least number of shifts.**")

    print("\nEnter number of employees:")
    num_employees = int(input())
    print("\nEnter number of shifts:")
    num_shifts = int(input())

print("\nScheduling", num_employees, "employees over", num_shifts, "shifts...\n")


Scheduling 7 employees over 4 shifts...



Another input data are the preferences of each employee for the shifts. In order to make things simpler the program will randomly generate employee preferences for the N shifts from most preferred (0) to least preferred (N-1).

In [3]:
# Generate random array of preferences over employees
preferences = np.tile(np.arange(num_shifts), (num_employees, 1))
rows = np.indices((num_employees,num_shifts))[0]
cols = [np.random.permutation(num_shifts) for _ in range(num_employees)]
preferences = preferences[rows, cols]
print(preferences)

[[2 1 0 3]
 [3 0 2 1]
 [0 1 3 2]
 [1 0 3 2]
 [0 3 1 2]
 [1 3 2 0]
 [3 1 2 0]]


### Formulate the problem: Building the DQM

The employee scheduling problem consists of two components: a requirement that employees are evenly distributed across all shifts, and an objective to satisfy employees by scheduling them for their preferred shifts.

To build the model we construct a Discrete Quadratic Models object [(see DQM class)](https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/models.html#discrete-quadratic-models) and add it a set of discrete variables that indicate the shift of each employee.

There are $M$ discrete variables that can take up to $N$ different values.

In [4]:
# Initialize the DQM object
dqm = DiscreteQuadraticModel()

# Build the DQM starting by adding variables
for name in range(num_employees):
    dqm.add_variable(num_shifts, label=name)   # labeled variables with num_shifts cases -> creation of num_employees variables
dqm.variables

Variables([0, 1, 2, 3, 4, 5, 6])

#### Preferred Shifts

Since shift preferences $p_{i,u}$ are ranked from most preferred (smallest value) to least preferred (largest value), the rankings can be used for the biases in the quadratic model. Since the hybrid solvers look for minimum energy solutions, and minimum rank corresponds to most preferred, these naturally align.

The objective function of our problem can be formulatrd as:

$\sum_i^M \sum_u^N p_{i,u}x_{i,u}$

#### Even Distribution

An even distribution of employees across shifts would have approximately num_employees/num_shifts $M/N$ scheduled employees per shift. To enforce this requirement, both linear and quadratic biases must be adjusted in a specific manner. We can do this using linear and quadratic biases to enforce the constraint in the discrete quadratic model.

To determine the linear and quadratic bias adjustments, we must consider the underlying binary variables in our model. For a DQM with $N$ shifts and $M$ employees, each employee has a single variable $d_i$ constructed with $N$ cases or classes. These are implemented as $N$ binary variables $x_{i,u}$ per employee — one for each possible shift.

For a specific shift $u$, we require that exactly $M/N$ employees are scheduled. This is equivalent to saying that $M/N$ employee variables are assigned case $u$, or, returning to our binary variables, that $M/N$ of the binary variables corresponding to case $u$ take a value of 1. In other words, the sum of all employee case $u$ binary variables should equal $M/N$. An equality over a summation of binary variables can be converted to a minimization expression by moving from the equality:

$\sum_{i=1}^M x_{i,u} = \frac{M}{N}  \ \ \ \ \
\forall u$

to a minimization expression:

$(\frac{M}{N} - \sum_{i=1}^M x_{i,u})^2    \ \ \ \ \  \forall u$

Expanding this expression of binary variables, it becomes:

$(\frac{M}{N})^2 - 2\frac{M}{N} \sum_{i=1}^M x_{i,u} + (\sum_{i=1}^M x_{i,u})^2  \ \ \ \ \   \forall u$

$= (\frac{M}{N})^2 - 2\frac{M}{N} \sum_{i=1}^M x_{i,u} + \sum_{i=1}^M x_{i,u} +2\sum_{i=1}^M \sum_{j>i}^M x_{i,u}x_{j,u}  \ \ \ \ \   \forall u$

Removing the constant and simplifying this expression of binary variables, it becomes:

$(1- 2\frac{M}{N}) \sum_{i=1}^M x_{i,u} + 2 \sum_{i=1}^M \sum_{j>i}^M x_{i,u}x_{j,u}  \ \ \ \ \   \forall u$


This expression give us $N$ terms for $N$ contrains $C_u$ that our DQN must satisfy in orther to fullfil it for each shift ($\forall u$).

The sum of all $N$ constrain terms $C_u$ is:

$C_u = \sum_{u=1}^N (1-2\frac{M}{N}) \sum_{i=1}^M x_{i,u} + 2 \sum_{u=1}^N \sum_{i=1}^M \sum_{j>i}^M x_{i,u}x_{j,u}$

When this constraint is built into our DQM object, it is added in with a coefficient gamma. This term gamma is known as a Lagrange parameter and can be used to weight this constraint against the competing employee preferences.

The final function that we need to optimize is the objective function plus the $N$ terms of the $C_u$ restrictions:

$H(x)=\sum_i^M \sum_u^N p_{i,u}x_{i,u} + \gamma \sum_u^N C_u$

$=\sum_i^M \sum_u^N (p_{i,u}+\gamma (1-2\frac{M}{N}))x_{i,u} + 2 \gamma \sum_{u=1}^N \sum_{i=1}^M \sum_{j>i}^M x_{i,u}x_{j,u}$

And comparing with the equation of the DQM we can identify the linear and quadratic terms:

$a_i=p_{i,u}- \gamma (2\frac{M}{N}-1)$

$b_{i,j}=2* \gamma$

Now we add the linear and the quadratic terms to the DQM object.

To do that more efficiently in the for loops we re-arrange the summations in the minimization expression as following:


$=\sum_u^N (\sum_i^M (p_{i,u}-\gamma (2\frac{M}{N}-1))x_{i,u} + 2 \gamma \sum_{j>i}^M x_{i,u}x_{j,u}))$

and we select the value of the Lagrange parameter $\gamma$ to be the number of employess $M$. This term gamma is known as a Lagrange parameter and can be used to weight this constraint against the competing employee preferences. You may wish to adjust this parameter depending on your problem requirements and size. The value set here in this program was chosen to empirically work well as a starting point for problems of a wide-variety of sizes.

In [ ]:
# Distribute employees equally across shifts according to preferences
num_per_shift = int(num_employees/num_shifts)
gamma = num_employees

for u in range(num_shifts):
    for i in range(num_employees):
        dqm.set_linear_case(i, u, preferences[i, u] - gamma*(2*num_per_shift-1))
        for j in range(i+1, num_employees):
            dqm.set_quadratic_case(i, u, j, u, 2*gamma)

### Solve the problem in a QPU with Leap Hybrid Solvers

To sample a DQM we use a DWave Hybrid solver, the [LeapHybridDQMSampler()](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html#id19) and use the SAPI to sumbmit the sampling job to the QPU

In [ ]:
# Initialize the DQM solver
sampler = LeapHybridDQMSampler()

# Solve the problem using the DQM solver
sampleset = sampler.sample_dqm(dqm, label='Example - Employee Scheduling')

We print the sampling results

In [ ]:
#get all solutions
print("\nSchedule first sample:", sampleset)

In [ ]:
# Get the first solution, and print it
sample = sampleset.first.sample
energy = sampleset.first.energy
print("\nSchedule first sample:", sample)
print("\nSchedule score:", energy)

### Final results

Once we have the results, we build two images to ilustrate the distibution of the employees in the shifts:

- First, `employee_schedule.png` illustrates the employee preference matrix alongside the schedule built.
- Second, `schedule_statistics.png` shows how many employees are scheduled for each shift, alongside a bar chart showing the employees' average preferences for the shifts for which they have been scheduled.

At the end of the program's run, two statistics are printed to the command-line:

- Schedule score provides the energy value of the best solution found.
- Average happiness is the average of the employee preference values for the shifts that they are scheduled.

A smaller average happiness score is better, and a score of 0.0 is a perfect score - everyone received their first choice shift.

In [ ]:
# Build schedule
schedule = np.ones((num_employees, num_shifts))*num_shifts
prefs = [0]*num_shifts
shifts = [0]*num_shifts
for key, val in sample.items():
    schedule[key,val]=preferences[key,val]
    prefs[preferences[key,val]] += 1
    shifts[val] += 1

# Show heatmap of preferences
cmap = plt.get_cmap('seismic')
cmaplist = [cmap(i) for i in range(cmap.N)]
cmaplist[-1] = (1.0,1.0,1.0,1.0)
cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(preferences, cmap='seismic', interpolation='nearest', vmin=0, vmax=num_shifts, aspect='auto')
ax1.xlabel = 'Shifts'
ax1.ylabel = 'Employees'
ax1.set_title("Employee Shift Preferences", color='Black', fontstyle='italic')

# Show heatmap of schedule
cax = ax2.imshow(schedule, cmap=cmap, interpolation='nearest', aspect='auto', vmin=0)
cbar = fig.colorbar(cax, ticks=[0, num_shifts])
cbar.set_ticklabels(['Best', 'Worst'])
ax2.xlabel = 'Shifts'
ax2.set_title("Employee Shift Schedule", color='Black', fontstyle='italic')
plt.savefig("employee_schedule.png")

# Compute/display schedule statistics
plt.clf()
plt.subplot(1, 2, 1)
plt.bar(np.arange(num_shifts), shifts)
plt.xlabel("Shift")
plt.ylabel("Number Scheduled")
plt.title("Employees Scheduled Per Shift")

mean_happiness = sum([i*prefs[i] for i in range(num_shifts)])/num_employees
print("\nAverage happiness:\t", mean_happiness)

plt.subplot(1, 2, 2)
plt.bar(np.arange(num_shifts), prefs)
plt.xlabel("Preference Rank")
plt.title("Average Preference per Shift")
plt.savefig("schedule_statistics.png")